In [ ]:
# This is necessary to recognize the modules
import os
import sys


root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)


In [ ]:
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from hummingbot.connector.connector_base import TradeType


def get_bt_candlestick_trace(df):
    df.index = pd.to_datetime(df.timestamp, unit='s')
    return go.Scatter(x=df.index,
                      y=df['close'],
                      mode='lines',
                      line=dict(color="blue"),
                      )


def get_pnl_trace(executors):
    pnl = [e.net_pnl_quote for e in executors]
    cum_pnl = np.cumsum(pnl)
    return go.Scatter(
        x=pd.to_datetime([e.close_timestamp for e in executors], unit="s"),
        y=cum_pnl,
        mode='lines',
        line=dict(color='gold', width=2, dash="dash"),
        name='Cumulative PNL'
    )


def get_default_layout(title=None, height=800, width=1800):
    layout = {
        "template": "plotly_dark",
        "plot_bgcolor": 'rgba(0, 0, 0, 0)',  # Transparent background
        "paper_bgcolor": 'rgba(0, 0, 0, 0.1)',  # Lighter shade for the paper
        "font": {"color": 'white', "size": 12},  # Consistent font color and size
        "height": height,
        "width": width,
        "margin": {"l": 20, "r": 20, "t": 50, "b": 20},
        "xaxis_rangeslider_visible": False,
        "hovermode": "x unified",
        "showlegend": False,
    }
    if title:
        layout["title"] = title
    return layout


def add_executors_trace(fig, executors, row, col):
    for executor in executors:
        entry_time = pd.to_datetime(executor.timestamp, unit='s')
        entry_price = executor.custom_info["current_position_average_price"]
        exit_time = pd.to_datetime(executor.close_timestamp, unit='s')
        exit_price = executor.custom_info["close_price"]
        name = "Buy Executor" if executor.config.side == TradeType.BUY else "Sell Executor"

        if executor.filled_amount_quote == 0:
            fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[entry_price, entry_price], mode='lines',
                                     line=dict(color='grey', width=2, dash="dash"), name=name), row=row, col=col)
        else:
            if executor.net_pnl_quote > Decimal(0):
                fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[entry_price, exit_price], mode='lines',
                                         line=dict(color='green', width=4), name=name), row=row, col=col)
            else:
                fig.add_trace(go.Scatter(x=[entry_time, exit_time], y=[entry_price, exit_price], mode='lines',
                                         line=dict(color='red', width=4), name=name), row=row, col=col)

    return fig


def create_backtesting_figure(df, executors, config):
    # Create subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.02, subplot_titles=('Candlestick', 'PNL Quote'),
                        row_heights=[0.7, 0.3])

    # Add candlestick trace
    fig.add_trace(get_bt_candlestick_trace(df), row=1, col=1)

    # Add executors trace
    fig = add_executors_trace(fig, executors, row=1, col=1)

    # Add PNL trace
    fig.add_trace(get_pnl_trace(executors), row=2, col=1)

    # Apply the theme layout
    layout_settings = get_default_layout(f"Trading Pair: {config['trading_pair']}")
    layout_settings["showlegend"] = False
    fig.update_layout(**layout_settings)

    # Update axis properties
    fig.update_xaxes(rangeslider_visible=False, row=1, col=1)
    fig.update_xaxes(row=2, col=1)
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="PNL", row=2, col=1)
    return fig

In [ ]:
import pickle

import pandas as pd
path = os.path.join(root_path, "research_notebooks", "xtreet_bb", "backtesting_results_ts_nuevo.pickle")

bt_results = pickle.load(open(path, 'rb'))

In [ ]:
import pandas as pd

results_df = pd.DataFrame([result["results"] for result in bt_results.values()])
results_df

In [ ]:
import plotly.express as px

# Create a new column with custom hover text
results_df['custom_hover_text'] = results_df.apply(lambda row: f"""
Pair: {row['config']['trading_pair']}
<br>Volume: {row['total_volume']}
<br>PNL: {row['net_pnl_quote']}
<br>Close types: {row['close_types']}

Config:
<br>Id: {row['config']['id']}
<br>BB Length: {row['config']['bb_length']}
<br>BB Std: {row['config']['bb_std']}
<br>Take profit: {row['config']['take_profit']}
<br>Stop loss: {row['config']['stop_loss']}
<br>Time limit: {row['config']['time_limit']}
<br>DCA Spreads: {row['config']['dca_spreads']}
<br>DCA Amounts: {row['config']['dca_amounts_pct']}
""", axis=1)

# Create the scatter plot with the custom hover text
fig = px.scatter(
    results_df,
    x="total_volume",
    y="net_pnl_quote",
    color="trading_pair",  # Color by trading_pair
    hover_data={"custom_hover_text": True},  # Show the custom hover text
    color_discrete_sequence=px.colors.qualitative.Plotly,  # Optional: use a specific color sequence
    title="Net PNL Quote vs. Total Volume by Trading Pair"
)

# Show the figure
fig.show()


In [ ]:
results_df.sort_values("net_pnl_quote", ascending=False)

In [ ]:
from pprint import pprint
from decimal import Decimal


result = bt_results[results_df.sort_values("net_pnl_quote", ascending=False).loc[138]["config"]["id"] + ".yml"]
df = result["df"]
config = result["config"]
executors = result["executors"]


fig = create_backtesting_figure(
    df=df,
    executors=executors,
    config=config.dict())
# df.ta.bbands(length=config.bb_length, std=config.bb_std, append=True)
fig.add_trace(go.Scatter(x=df.index,
                         y=df[f"BBU_{config.bb_length}_{config.bb_std}"],
                         line=dict(color='lightblue', width=1))
              )
# fig.add_trace(go.Scatter(x=df.index,
#                          y=df[f"BBM_{config.bb_length}_{config.bb_std}"])
#               )
fig.add_trace(go.Scatter(x=df.index,
                         y=df[f"BBL_{config.bb_length}_{config.bb_std}"],
                         line=dict(color='lightblue', width=1))
                         )
pprint(config)
fig.update_layout(width=1400)
fig.show()

In [ ]:
executors

In [ ]:
filtered_report = results_df[results_df["net_pnl_quote"] > 20]
unique_trading_pairs = filtered_report["trading_pair"].unique()

In [ ]:
best_configs = []
for trading_pair in unique_trading_pairs:
    best_configs.append(filtered_report[filtered_report["trading_pair"] == trading_pair].sort_values("net_pnl_quote", ascending=False).iloc[0]["config"])

In [ ]:
from services.backend_api_client import BackendAPIClient

client = BackendAPIClient(host="localhost")

In [ ]:
for config in best_configs:
    if "position_mode" in config:
        config.pop("position_mode")
    if "take_profit_order_type" in config:
        config.pop("take_profit_order_type")
    client.add_controller_config(config)

In [ ]:
from decimal import Decimal
configs_id = [config["id"] + ".yml" for config in best_configs if (config["trading_pair"] != "CRV-USDT") and (config["dca_amounts_pct"][1] > Decimal(1)) and (config["dca_spreads"][1] > Decimal(0.0001))]

In [ ]:
configs_id

In [ ]:
client.deploy_script_with_controllers(
    bot_name="la_maquinita",
    controller_configs=configs_id,
    credentials="master_account",
    time_to_cash_out=200000,
    max_global_drawdown=50,
    max_controller_drawdown=15
)